In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

from utils import val_to_label, label_to_val, get_device

In [2]:
path = "IMDB Dataset.csv"

df = pd.read_csv(path)
texts = df['review'].tolist()
labels = df["sentiment"].apply(val_to_label).to_list()

device = get_device()

In [3]:
n = 3000
train_texts, test_texts, train_labels, test_labels = train_test_split(texts[:n], labels[:n], test_size=0.2, random_state=42)
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=0.2, random_state=42)

In [4]:
# MLP
from mlp import run as run_mlp

bert_model_name = 'bert-base-uncased'
num_classes = 2
max_length = 128
batch_size = 16
num_epochs = 4
learning_rate = 2e-5

report_mlp = run_mlp(train_texts, train_labels, val_texts, val_labels, test_texts, test_labels, 
                     bert_model_name, num_classes, max_length, batch_size, num_epochs, learning_rate)

/Users/taakoah1/micromamba/envs/hftuner/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/taakoah1/micromamba/envs/hftuner/lib/python3.11/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/4


100%|██████████| 30/30 [00:03<00:00,  8.44it/s]


Validation Accuracy: 0.8375
Epoch 2/4


100%|██████████| 30/30 [00:03<00:00,  8.46it/s]


...

Validation Accuracy: 0.8583
Epoch 4/4


100%|██████████| 30/30 [00:03<00:00,  8.43it/s]


Validation Accuracy: 0.8458


100%|██████████| 38/38 [00:04<00:00,  8.35it/s]

              precision    recall  f1-score   support

           0       0.89      0.83      0.86       316
           1       0.82      0.88      0.85       284

    accuracy                           0.85       600
   macro avg       0.85      0.85      0.85       600
weighted avg       0.86      0.85      0.85       600

/nrun executed in 3.4414 min.


In [14]:
from zeroshot import run as run_zeroshot

model_name = "facebook/bart-large-mnli"
batch_size = 8
candidate_labels = ['negative', 'positive']

report_zeroshot = run_zeroshot(test_texts, test_labels, model_name, candidate_labels, batch_size)

 21%|██▏       | 16/75 [01:26<05:17,  5.38s/it]


KeyboardInterrupt: 

# Unsupervised

### Text generation

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2', low_cpu_mem_usage=True).to(device)

In [ ]:
prompt = "The Belgian national football team "
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

generated_ids = model.generate(input_ids, do_sample=True, temperature=0.9, max_length=200)
generated_text = tokenizer.decode(generated_ids[0])
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The Belgian national football team  recently finished the quarter-finals with a 3-2 win over Real Madrid to give them a place in the Champions League.
Giovanni Sforza's side won the league title.
Giovanni Sforza's side won the league title. 
Giovanni Sforza's side won the league title. 
Giovanni Sforza's side won the league title.
Giovanni Sforza's side won the league title. 
Giovanni Sforza's side won the league title. 
Giovanni Sforza's side won the league title.
Giovanni Sforza's side won the league title.
This is a story about a boy named Lorenzo who has only recently become a footballer and not a footballer at all.
His dad, Lorenzo, is a top scorer in the Pro12 tournament and is a good friend of Vincenzo Montella


In [ ]:
prompt = """
Classify the following sentences as "positive" or "negative".

Sentence: This movie is very nice.
Sentiment: positive

#####

Sentence: I hated this movie, it sucks.
Sentiment: negative

#####

Sentence: This movie was actually pretty funny.
Sentiment: positive

#####

Sentence: This movie could have been better.
Sentiment: """
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

generated_ids = model.generate(input_ids, do_sample=True, temperature=0.1, max_length=200)
generated_text = tokenizer.decode(generated_ids[0])
print(generated_text)
     

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Classify the following sentences as "positive" or "negative".

Sentence: This movie is very nice.
Sentiment: positive

#####

Sentence: I hated this movie, it sucks.
Sentiment: negative

#####

Sentence: This movie was actually pretty funny.
Sentiment: positive

#####

Sentence: This movie could have been better.
Sentiment: ive never seen this movie before.

Sentiment: ive never seen this movie before.

Sentiment: ive never seen this movie before.

Sentiment: ive never seen this movie before.

Sentiment: ive never seen this movie before.

Sentiment: ive never seen this movie before.

Sentiment: ive never seen this movie before.

Sentiment: ive never seen this movie before.

Sentiment: ive never


In [8]:
from transformers import pipeline

# see https://github.com/huggingface/transformers/blob/ad697f18016cf5eb4be48f9552c69c2421aa5581/src/transformers/generation/configuration_utils.py#L233

prompt = """
Classify the following sentences as "positive" or "negative".

Sentence: This movie is very nice.
Sentiment: positive

#####

Sentence: I hated this movie, it sucks.
Sentiment: negative

#####

Sentence: This movie was actually pretty funny.
Sentiment: positive

#####

Sentence: This movie could have been better.
Sentiment: """

model_name = "microsoft/Phi-3-mini-4k-instruct"

generator = pipeline('text-generation', model=model_name, trust_remote_code=True)
text = generator(prompt, max_length=len(prompt)+12)
print(text[0]["generated_text"])

ValueError: Could not load model microsoft/Phi-3-mini-4k-instruct with any of the following classes: (<class 'transformers.models.auto.modeling_auto.AutoModelForCausalLM'>,). See the original errors:

while loading with AutoModelForCausalLM, an error is thrown:
Traceback (most recent call last):
  File "/Users/taakoah1/micromamba/envs/hftuner/lib/python3.11/site-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/taakoah1/micromamba/envs/hftuner/lib/python3.11/site-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/Users/taakoah1/micromamba/envs/hftuner/lib/python3.11/site-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 61] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/taakoah1/microm

In [6]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM
from utils import get_device


device = get_device()

model_name = "microsoft/Phi-3-mini-4k-instruct-gguf"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True).to(device)

input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

OSError: microsoft/Phi-3-mini-4k-instruct-gguf does not appear to have a file named config.json. Checkout 'https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/main' for available files.

In [53]:
generated_ids = model.generate(input_ids, do_sample=True, temperature=0.1, max_length=len(prompt)+10)
generated_text = tokenizer.decode(generated_ids[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [58]:
from transformers import pipeline
import torch

generator = pipeline('text-generation', model = 'openai-community/gpt2')
prompt = "Hello, I'm a language model"

generator(prompt, max_length = 30, temperature=0.8)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "Hello, I'm a language modeler, and so I wanted to know how the compiler handles different types, so I created a test for both of"}]

In [7]:
from llama_cpp import Llama


llm = Llama(
  model_path="./Phi-3-mini-4k-instruct-q4.gguf",  # path to GGUF file
  n_ctx=4096,  # The max sequence length to use - note that longer sequence lengths require much more resources
  n_threads=8, # The number of CPU threads to use, tailor to your system and the resulting performance
  n_gpu_layers=35, # The number of layers to offload to GPU, if you have GPU acceleration available. Set to 0 if no GPU acceleration is available on your system.
)

prompt = "How to explain Internet to a medieval knight?"

# Simple inference example
output = llm(
  f"<|user|>\n{prompt}<|end|>\n<|assistant|>",
  max_tokens=256,  # Generate up to 256 tokens
  stop=["<|end|>"], 
  echo=True,  # Whether to echo the prompt
)

print(output['choices'][0]['text'])

ValueError: Model path does not exist: ./Phi-3-mini-4k-instruct-q4.gguf